<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [1]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [2]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
os.environ["HOPSWORKS_API_KEY"] = "c360klTVehII8xXY.u3NgrKOFiZgjt7SoRKxdWLivKdWkO13v87yBjmGvtdI69MBbCP6mYfKiVGbtsU2I"

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1169561
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "germany", "city": "rheinlandpfalz", "street": "Mainz-Parcusstra\\u00dfe", "aqicn_url": "https://api.waqi.info/feed/@6135", "latitude": "50.0002", "longitude": "8.1538"}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [3]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

In [4]:
air_quality_fg.select(["date", "pm25"]).show(10)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.81s) 


,date,pm25
0,2014-10-03 00:00:00+00:00,61.0
1,2018-06-09 00:00:00+00:00,51.0
2,2015-07-07 00:00:00+00:00,33.0
3,2023-03-15 00:00:00+00:00,13.0
4,2014-10-07 00:00:00+00:00,42.0
5,2023-12-22 00:00:00+00:00,9.0
6,2015-10-03 00:00:00+00:00,57.0
7,2015-06-09 00:00:00+00:00,24.0
8,2016-02-01 00:00:00+00:00,16.0
9,2018-06-05 00:00:00+00:00,32.0


---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [5]:
import requests
import pandas as pd
from datetime import timezone
from hsfs.feature import Feature
import calendar

one_day_ago = (datetime.date.today() - datetime.timedelta(days=1))
two_day_ago = (datetime.date.today() - datetime.timedelta(days=2))
three_day_ago = (datetime.date.today() - datetime.timedelta(days=3))

one_day_ago = datetime.datetime(one_day_ago.year, one_day_ago.month, one_day_ago.day)
two_day_ago = datetime.datetime(two_day_ago.year, two_day_ago.month, two_day_ago.day)
three_day_ago = datetime.datetime(three_day_ago.year, three_day_ago.month, three_day_ago.day)

#one_day_ago = one_day_ago.replace(tzinfo=timezone.utc)
#two_day_ago = two_day_ago.replace(tzinfo=timezone.utc)
#three_day_ago = three_day_ago.replace(tzinfo=timezone.utc)
print(one_day_ago)
print(datetime.datetime.today())

df_one_ago = air_quality_fg.select(['date','pm25']).filter(air_quality_fg.date == one_day_ago).show(1)
df_two_ago = air_quality_fg.select(['date','pm25']).filter(air_quality_fg.date == two_day_ago).show(1)
df_three_ago = air_quality_fg.select(['date','pm25']).filter(air_quality_fg.date == three_day_ago).show(1)

#print(air_quality_fg.select(['date', 'pm25']).filter((air_quality_fg.date < one_day_ago) & (air_quality_fg.date > two_day_ago)).show(1))
#print(air_quality_fg.select(['date','pm25']).filter(air_quality_fg.date == two_day_ago).show(1))
#print(air_quality_fg.select(['date','pm25']).filter(air_quality_fg.date == three_day_ago).show(1))

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df["pm25_one_day_ago"] = df_one_ago["pm25"]
aq_today_df["pm25_two_day_ago"] = df_two_ago["pm25"]
aq_today_df["pm25_three_day_ago"] = df_three_ago["pm25"]
aq_today_df

2024-11-19 00:00:00
2024-11-20 11:55:53.241767
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.69s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.85s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.09s) 


,pm25,country,city,street,date,url,pm25_one_day_ago,pm25_two_day_ago,pm25_three_day_ago
0,5.0,germany,rheinlandpfalz,Mainz-Parcusstraße,2024-11-20,https://api.waqi.info/feed/@6135,5.0,5.0,59.0


In [6]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pm25                1 non-null      float32       
 1   country             1 non-null      object        
 2   city                1 non-null      object        
 3   street              1 non-null      object        
 4   date                1 non-null      datetime64[ns]
 5   url                 1 non-null      object        
 6   pm25_one_day_ago    1 non-null      float32       
 7   pm25_two_day_ago    1 non-null      float32       
 8   pm25_three_day_ago  1 non-null      float32       
dtypes: datetime64[ns](1), float32(4), object(4)
memory usage: 184.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [7]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 50.0°N 8.25°E
Elevation 163.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-20,4.50,0.3,22.705700,267.273743,rheinlandpfalz
1,2024-11-21,2.65,0.0,11.966954,223.781204,rheinlandpfalz
2,2024-11-22,3.00,0.0,22.473343,234.782318,rheinlandpfalz
3,2024-11-23,4.60,0.0,14.512064,203.385239,rheinlandpfalz
4,2024-11-24,15.35,0.0,15.790833,204.227737,rheinlandpfalz
5,2024-11-25,12.10,0.0,5.483356,113.198624,rheinlandpfalz
6,2024-11-26,11.05,0.0,17.848160,228.270584,rheinlandpfalz
7,2024-11-27,9.00,0.0,15.175612,247.693741,rheinlandpfalz
8,2024-11-28,5.80,0.0,2.099143,120.963692,rheinlandpfalz
9,2024-11-29,6.45,0.3,8.280000,180.000000,rheinlandpfalz


In [8]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [9]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2024-11-20 11:55:59,176 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1169561/fs/1160264/fg/1351119


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1169561/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1f87ade8700>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 675003
         }
       },
       "result": {
         "observed_value": 5.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-20T10:55:59.000176Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "unsuc

In [10]:
# Insert new data
weather_fg.insert(daily_df)

2024-11-20 11:56:08,809 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1169561/fs/1160264/fg/1351120


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1169561/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1f87ade98d0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 675004
         }
       },
       "result": {
         "observed_value": 2.099142551422119,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-20T10:56:08.000809Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
